In [7]:
import numpy as np
import matplotlib.pyplot as plt
from ansys.mapdl.core import launch_mapdl
mapdl=launch_mapdl()
print(mapdl)

Product:             Ansys Mechanical Enterprise Academic Student
MAPDL Version:       23.1
ansys.mapdl Version: 0.64.0



In [17]:
mapdl.clear()
mapdl.prep7()


*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   08:51:05  APR 19, 2023 CP=      5.000

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [18]:
# Define Material Property
E = 210e9
nu = 0.3
YS = 240e6
# Defining Geometry
a1 = 0.02 # Crack length
a2 = 0.02
c = 0.03 # Thickness
theta1 = np.pi/20
theta2 = -np.pi/20
x = 0.005
shift = 0.02

In [19]:
# Define Material Property

    # Defining Geometry 
    # Corners of the rectangle
m0 = mapdl.k("", 0,-0.1)
m1 = mapdl.k("",0,0.1)
m2 = mapdl.k("",x-0.0025,0.1)
m3 = mapdl.k("",x-0.0025,0.1002)
m4 = mapdl.k("",x+0.0025,0.1002)
m5 =mapdl.k("",x+0.0025,0.1)
m6 = mapdl.k("", 0.1, 0.1)
m7 = mapdl.k("", 0.1, -0.1)
main = mapdl.a(m0,m1,m2,m3,m4,m5,m6,m7)

# Defining points for the crack
b = 0.0001
beta = np.pi/12
us1 = np.tan(theta1)
us2 = np.tan(theta1+beta)# lower side of crack 1
us3 = np.tan(theta1-beta)# upper side of crack 2

d = a1*(np.sin(theta1)-np.cos(theta1)*us2)
e = a1*(np.sin(theta1)-np.cos(theta1)*us3)

x1 = (b+d)/(us1-us2)
y1 = us1*x1-b 
x3 = (b-e)/(us3-us1)
y3 = us1*x3+b 



In [20]:

# Crack geometry for Crack 1
k0 = mapdl.k("",0,-b +shift)
k1 = mapdl.k("",x1,y1+shift)
k2 = mapdl.k("",a1*np.cos(theta1),a1*np.sin(theta1)+shift)
k3 = mapdl.k("",x3,y3+shift)
k4 = mapdl.k("",0,b+shift)
crack1 = mapdl.a(k0,k1,k2,k3,k4)
aout = mapdl.asba(main, crack1)

### Geometry for Crack 2

ls1 = np.tan(theta2)
ls2 = np.tan(theta2+beta)# lower side of crack 1
ls3 = np.tan(theta2-beta)# upper side of crack 2

d = a2*(np.sin(theta2)-np.cos(theta2)*ls2)
e = a2*(np.sin(theta2)-np.cos(theta2)*ls3)

x1 = (b+d)/(ls1-ls2)
y1 = ls1*x1-b 
x3 = (b-e)/(ls3-ls1)
y3 = ls1*x3+b 


k0 = mapdl.k("",0,-b -shift)
k1 = mapdl.k("",x1,y1-shift)
k2 = mapdl.k("",a2*np.cos(theta2),a2*np.sin(theta2)-shift)
k3 = mapdl.k("",x3,y3-shift)
k4 = mapdl.k("",0,b-shift)
crack2 = mapdl.a(k0,k1,k2,k3,k4)
aout = mapdl.asba(aout, crack2)

mapdl.vext(aout, dz=0.005)
mapdl.vplot(show_lines=True, quality=5)

Widget(value="<iframe src='http://localhost:52796/index.html?ui=P_0x28c4f4c5c60_4&reconnect=auto' style='width…

In [21]:
mapdl.et(1,"SOLID285")
    # Define the Mesh
mapdl.smrtsize(1)
mapdl.aesize("ALL", 0.003)
mapdl.vmesh(1)

mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")

Widget(value="<iframe src='http://localhost:52796/index.html?ui=P_0x28c4cbddde0_5&reconnect=auto' style='width…

In [22]:
# Using the SI units
press= 250e5
mapdl.units("SI")  # SI - International system (m, kg, s, K).

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio
# Fix the lower side.
mapdl.nsel("S", "LOC", "Y", -0.1)
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")
mapdl.d("ALL", "UX")

mapdl.nsel("S", "LOC", "Y", 0.1)
mapdl.cp(5, "UX", "ALL")

mapdl.asel("S",'LOC',"Y",0.1002)
mapdl.sfa('ALL', '', 'PRES', -press)
mapdl.allsel(mute=True)
mapdl.run("/SOLU")
mapdl.antype("STATIC")
mapdl.solve()
mapdl.finish(mute=True)

# List parameters to post-process
mapdl.post1()





*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   08:51:16  APR 19, 2023 CP=      9.734

                                                                               



          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

In [14]:
mapdl.path('P1',3,5)
node1 = mapdl.queries.node(a1*np.cos(theta1),a1*np.sin(theta1)+shift,0)
xs1 = 0.6*a1
ys1 = us1*xs1-b+shift
node2 = mapdl.queries.node(xs1,ys1,0)
node3 = mapdl.queries.node(0,-b+shift,0)
mapdl.ppath(1, node1)
mapdl.ppath(2, node2)
mapdl.ppath(3, node3)
mapdl.local(11,0,a1*np.cos(theta1),a1*np.sin(theta1)+shift,0,theta*180/np.pi)


NameError: name 'theta' is not defined

In [23]:
#mapdl.csys(11)
#k1 = mapdl.kcalc(0,1,0,0)
result = mapdl.result
result.plot_principal_nodal_stress(
    0,
    "SEQV",
    lighting=False,
    background="w",
    show_edges=True,
    text_color="k",
    add_text=False,
)


Widget(value="<iframe src='http://localhost:52796/index.html?ui=P_0x28c4a5b13f0_6&reconnect=auto' style='width…

In [152]:
k1

****  CALCULATE MIXED-MODE STRESS INTENSITY  FACTORS  ****

   ASSUME PLANE STRAIN CONDITIONS

   ASSUME A HALF-CRACK MODEL WITH SYMMETRY BOUNDARY CONDITIONS (USE 3 NODES)

   EXTRAPOLATION PATH IS DEFINED BY NODES:       66      63      45
    WITH NODE      66 AS THE CRACK-TIP NODE

   USE MATERIAL PROPERTIES FOR MATERIAL NUMBER     1
    EX =    0.21000E+12   NUXY =    0.30000      AT TEMP =  0.0000    

 ****  KI =  0.61167E+08,   KII =   0.0000    ,   KIII =   0.0000      ****

In [153]:
mapdl.clear()
mapdl.prep7()

*** MAPDL - ENGINEERING ANALYSIS SYSTEM  RELEASE 2023 R1          23.1     ***
 Ansys Mechanical Enterprise Academic Student      
 01055371  VERSION=WINDOWS x64   20:10:43  APR 16, 2023 CP=      5.312

                                                                               



          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

In [154]:

# Define Material Property
E = 210e9
nu = 0.3
YS = 240e6
# Defining Geometry
a1 = 0.02 # Crack length
a2 = 0.02
c = 0.03 # Thickness
theta1 = np.pi/20
theta2 = -np.pi/20
x = 0.005
shift = 0.02


E = 210e9
nu = 0.3
YS = 240e6

    # Defining Geometry 
    # Corners of the rectangle
m0 = mapdl.k("", 0,-0.1)
m1 = mapdl.k("",0,0.1)
m2 = mapdl.k("",x-0.0025,0.1)
m3 = mapdl.k("",x-0.0025,0.1002)
m4 = mapdl.k("",x+0.0025,0.1002)
m5 =mapdl.k("",x+0.0025,0.1)
m6 = mapdl.k("", 0.1, 0.1)
m7 = mapdl.k("", 0.1, -0.1)
main = mapdl.a(m0,m1,m2,m3,m4,m5,m6,m7)

# Defining points for the crack
b = 0.0001
beta = np.pi/12
us1 = np.tan(theta1)
us2 = np.tan(theta1+beta)# lower side of crack 1
us3 = np.tan(theta1-beta)# upper side of crack 2

d = a1*(np.sin(theta1)-np.cos(theta1)*us2)
e = a1*(np.sin(theta1)-np.cos(theta1)*us3)

x1 = (b+d)/(us1-us2)
y1 = us1*x1-b 
x3 = (b-e)/(us3-us1)
y3 = us1*x3+b 



# Crack geometry for Crack 1
k0 = mapdl.k("",0,-b +shift)
k1 = mapdl.k("",x1,y1+shift)
k2 = mapdl.k("",a1*np.cos(theta1),a1*np.sin(theta1)+shift)
k3 = mapdl.k("",x3,y3+shift)
k4 = mapdl.k("",0,b+shift)
crack1 = mapdl.a(k0,k1,k2,k3,k4)
aout = mapdl.asba(main, crack1)

### Geometry for Crack 2

ls1 = np.tan(theta2)
ls2 = np.tan(theta2+beta)# lower side of crack 1
ls3 = np.tan(theta2-beta)# upper side of crack 2

d = a2*(np.sin(theta2)-np.cos(theta2)*ls2)
e = a2*(np.sin(theta2)-np.cos(theta2)*ls3)

x1 = (b+d)/(ls1-ls2)
y1 = ls1*x1-b 
x3 = (b-e)/(ls3-ls1)
y3 = ls1*x3+b 


k0 = mapdl.k("",0,-b -shift)
k1 = mapdl.k("",x1,y1-shift)
k2 = mapdl.k("",a2*np.cos(theta2),a2*np.sin(theta2)-shift)
k3 = mapdl.k("",x3,y3-shift)
k4 = mapdl.k("",0,b-shift)
crack2 = mapdl.a(k0,k1,k2,k3,k4)
aout = mapdl.asba(aout, crack2)

mapdl.vext(aout, dz=0.005)
mapdl.vplot(show_lines=True, quality=5)


mapdl.et(1,"SOLID285")
    # Define the Mesh
mapdl.smrtsize(1)
mapdl.aesize("ALL", 0.003)
mapdl.vmesh(1)

mapdl.eplot(vtk=True, show_edges=True, show_axes=False, line_width=2, background="w")


# Using the SI units
press= 250e6
mapdl.units("SI")  # SI - International system (m, kg, s, K).

# Define a material (nominal steel in SI)
mapdl.mp("EX", 1, E)  # Elastic moduli in Pa (kg/(m*s**2))
mapdl.mp("DENS", 1, 7800)  # Density in kg/m3
mapdl.mp("NUXY", 1, nu)  # Poisson's Ratio
# Fix the lower side.
mapdl.nsel("S", "LOC", "Y", -0.1)
mapdl.d("ALL", "UY")
mapdl.d("ALL", "UZ")
mapdl.d("ALL", "UX")

mapdl.nsel("S", "LOC", "Y", 0.1)
mapdl.cp(5, "UX", "ALL")

mapdl.asel("S",'LOC',"Y",0.1002)
mapdl.sfa('ALL', '', 'PRES', -press)
mapdl.allsel(mute=True)
mapdl.run("/SOLU")
mapdl.antype("STATIC")
mapdl.solve()
mapdl.finish(mute=True)

# List parameters to post-process
mapdl.post1()

mapdl.path('P2',3,5)
node4 = mapdl.queries.node(a*np.cos(theta2),a*np.sin(theta2)-shift,0)
xs2 = 0.6*a2
ys2 = ls1*xs2+b-shift
node5 = mapdl.queries.node(xs2,ys2,0)
node6 = mapdl.queries.node(0,b-shift,0)
mapdl.ppath(1, node4)
mapdl.ppath(2, node5)
mapdl.ppath(3, node6)
mapdl.local(12,0,a2*np.cos(theta2),a2*np.sin(theta2)-shift,0,theta2*180/np.pi)
mapdl.post1()
mapdl.csys(12)
k2 = mapdl.kcalc(0,1,0,0)

result = mapdl.result

Widget(value="<iframe src='http://localhost:52944/index.html?ui=P_0x1d98cf51090_33&reconnect=auto' style='widt…

Widget(value="<iframe src='http://localhost:52944/index.html?ui=P_0x1d98cf85120_34&reconnect=auto' style='widt…

In [155]:
k2

****  CALCULATE MIXED-MODE STRESS INTENSITY  FACTORS  ****

   ASSUME PLANE STRAIN CONDITIONS

   ASSUME A HALF-CRACK MODEL WITH SYMMETRY BOUNDARY CONDITIONS (USE 3 NODES)

   EXTRAPOLATION PATH IS DEFINED BY NODES:       36      41      38
    WITH NODE      36 AS THE CRACK-TIP NODE

   USE MATERIAL PROPERTIES FOR MATERIAL NUMBER     1
    EX =    0.21000E+12   NUXY =    0.30000      AT TEMP =  0.0000    

 ****  KI =  0.76289E+08,   KII =   0.0000    ,   KIII =   0.0000      ****